In [1]:
import pandas as pd
import sys
import numpy as np
sys.path.append('../')


import nlp
import wrangle

import nltk

from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore")

In [2]:
pd.set_option('display.max_columns', None)
datadf, dictionarydf = wrangle.wrangle_data(path_prefix='../')

In [3]:
datadf

,job_title,job_taxo,job_id,job_conduct_res,job_analyze_res,job_buy_res_report,job_manage_res_proj,job_observe_res,job_plan_res,job_teach_res,job_advocate_res,job_hire_res_vendor,job_lead_res_team,num_employees,num_researchers,primary_industry,types_res_used,future_res,exp_conduct_res,exp_analyze_res,exp_buy_res_report,exp_manage_res_proj,exp_observe_res,exp_plan_res,exp_teach_res,exp_advocate_res,exp_hire_res_vendor,exp_lead_res_team,research_educ,research_educ_cat,research_educ_desc,learning_talks,learning_read,learning_meetup,learning_workshop,learning_conference,likely_watch_video,likely_internet,likely_book,likely_online_group,likely_colleague,likely_meetup,likely_conference,likely_workshop,factor_speaker,factor_diverse_speak,factor_topics,factor_format_sessions,factor_size,factor_network,factor_variety_attend,factor_code,factor_location,factor_ability_to_pay,how_pick_events,best_event,events_attend_recent,ideal_conference_size,ideal_conference_size_cat_orig,ideal_conference_size_cat,ideal_structure,ideal_structure_cat,ideal_multi_track,ideal_single_track,ideal_unconference,session_keynote,session_long_talk,session_short_talk,session_workshop,session_network,session_social_event,session_qa,sesson_topic_tables,other_conference_types,ideal_topics,ideal_attendees,recommendations,persona_id
resp_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
284,Academic,4,1.0,3,2,0,3,3,3,3,2,1,4,0.0,2.0,maritime,both qualitative and quantitative research,both qualitative and quantitative research,5,5,5,5,5,5,5,5,5,5,"sort of, psychology background.",1.0,NaN,0,1,1,2,3,1,3,3,2,3,3,4,4,3,3,4,2,3,1,3,1,3,4,NaN,NaN,"CHI, INTERACT",depending on the professional values,7.0,7.0,"any, depending on the values",5.0,0,0,0,4,2,2,2,3,2,2,2,NaN,research-practice connections,NaN,research-practice connections,2
288,Academic,4,1.0,1,2,1,3,3,3,3,3,0,3,4.0,3.0,Higher education,Multiple,Multiple,5,5,5,5,5,5,5,5,2,5,"Yes, I was taught how to conduct research",1.0,Research methods and statistics training in bo...,2,3,2,2,2,2,3,3,2,3,2,2,2,3,3,4,3,4,2,3,3,2,4,"Content, cost, timing (can I get away at this ...","Dagstuhl workshop - small, intense, intimate g...",Hawaii international conference on Education,I would attend any of these,6.0,6.0,Multi-track: Every attendee chooses their own ...,2.0,1,0,0,3,3,3,3,3,1,1,2,NaN,NaN,NaN,Don’t create schedule conflicts with other rel...,2
294,Academic,4,1.0,3,3,1,3,1,3,3,3,1,3,4.0,3.0,University,"User research / Design research\n\nInterviews,...",Online FG,4,4,3,3,4,4,4,4,3,3,"Yes, I was taught how to conduct research",1.0,BA Sociology\nMA LIS\nPhD. LIS\n\nonline cours...,2,3,2,2,2,3,4,3,3,3,3,3,3,3,3,4,4,3,4,3,3,4,3,- time for travel (having two small children]\...,Courses at Copenhagen University (deep knowle...,"Research Tuesdays (Czech), UX/Design Conferen...",It depends - I really enjoyed small conference...,7.0,7.0,It depends,5.0,0,0,0,2,2,2,2,2,2,2,2,NaN,NaN,NaN,Looking forward! \n,2
281,Academic,4,1.0,3,4,0,2,3,2,0,1,0,0,2.0,1.0,Ed Tech,"surveys, interviews, focus groups, secondary d...","surveys, interviews, focus groups, secondary d...",1,1,0,1,1,1,0,1,0,0,"Yes, I was taught how to conduct research",1.0,"I have an Applied Anthropology, MA. I designed...",1,2,0,1,2,3,4,3,2,4,2,3,3,2,3,3,3,1,1,1,2,3,4,My professional development budget allows me t...,A workshop at the Society for Applied Anthropo...,A local education implementation conference.,Large conference: 500+ attendees,4.0,4.0,Multi-track: Every attendee chooses their own ...,2.0,1,0,0,4,2,3,3,2,2,3,2,NaN,"Research design, innovative methods, communica...",NaN,NaN,2
280,Academic,4,1.0,3,3,0,1,1,3,2,1,0,0,NaN,NaN,education,NaN,NaN,4,4,0,2,2,3,2,3,0,0,"No, I was not taught how to conduct research",0.0,NaN,1,1,1,1,2,3,3,3,1,2,1,2,1,3,1,4,2,3,1,2,2,4,4,NaN,NaN,NaN,Large conference: 500+ attendees,4.0,4.0,Multi-track: Every attendee chooses their own ...,2.0,1,0,0,4,2,3,3,2,1,2,2,You should consider the combination of confere...,NaN,

In [4]:
datadf.describe()

,job_conduct_res,job_analyze_res,job_buy_res_report,job_manage_res_proj,job_observe_res,job_plan_res,job_teach_res,job_advocate_res,job_hire_res_vendor,job_lead_res_team,exp_conduct_res,exp_analyze_res,exp_buy_res_report,exp_manage_res_proj,exp_observe_res,exp_plan_res,exp_teach_res,exp_advocate_res,exp_hire_res_vendor,exp_lead_res_team,learning_talks,learning_read,learning_meetup,learning_workshop,learning_conference,likely_watch_video,likely_internet,likely_book,likely_online_group,likely_colleague,likely_meetup,likely_conference,likely_workshop,factor_speaker,factor_diverse_speak,factor_topics,factor_format_sessions,factor_size,factor_network,factor_variety_attend,factor_code,factor_location,factor_ability_to_pay,ideal_multi_track,ideal_single_track,ideal_unconference,session_keynote,session_long_talk,session_short_talk,session_workshop,session_network,session_social_event,session_qa,sesson_topic_tables
count,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000
mean,2.856749,2.913223,0.402204,2.600551,1.982094,2.931129,1.845730,2.924242,0.776860,1.860882,3.283747,3.323691,0.960055,2.633609,3.084022,3.139118,1.862259,2.906336,1.326446,1.632231,2.074380,2.659780,1.801653,1.451791,1.663912,2.982094,3.500000,3.263085,2.079890,3.053719,2.202479,2.250689,2.360882,2.236915,2.871901,3.652893,2.776860,2.209366,1.998623,2.028926,1.903581,2.944904,3.336088,0.625344,0.334711,0.061983,2.706612,2.754821,2.341598,2.988981,2.173554,1.480716,2.121212,2.431129
std,1.025941,0.939210,0.720686,1.166285,1.134273,0.950177,1.236637,1.056235,0.914143,1.486931,1.571974,1.553165,1.608530,1.702955,1.694565,1.589165,1.684913,1.711475,1.654099,1.674476,0.791318,0.571707,0.836609,0.784165,0.761119,0.960436,0.711967,0.834391,1.213355,0.983242,1.150254,1.143723,1.101378,1.157435,1.071954,0.590183,0.926135,1.168658,1.107341,1.158115,1.263411,1.019666,0.971309,0.484368,0.472215,0.241292,1.124344,0.977229,1.106273,1.109154,1.219458,1.239149,1.179908,1.211472
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,3.000000,0.000000,2.000000,1.000000,3.000000,1.000000,3.000000,0.000000,0.000000,2.000000,2.000000,0.000000,1.000000,2.000000,2.000000,0.000000,1.000000,0.000000,0.000000,2.000000,2.000000,1.000000,1.000000,1.000000,3.000000,3.000000,3.000000,1.000000,3.000000,1.000000,1.000000,2.000000,1.000000,2.000000,3.000000,2.000000,1.000000,1.000000,1.000000,1.000000,2.000000,3.000000,0.000000,0.000000,0.000000,2.000000,2.000000,2.000000,2.000000,1.000000,0.000000,1.000000,2.000000
50%,3.000000,3.000000,0.000000,3.000000,2.000000,3.000000,2.000000,3.000000,1.000000,2.000000,4.000000,4.000000,0.000000,2.000000,3.000000,3.000000,1.000000,3.000000,1.000000,1.000000,2.000000,3.000000,2.000000,1.000000,2.000000,3.000000,4.000000,3.000000,2.000000,3.000000,2.000000,2.000000,2.000000,2.000000,3.000000,4.000000,3.000000,2.000000,2.000000,2.000000,2.000000,3.000000,4.000000,1.000000,0.000000,0.000000,3.000000,3.000000,3.000000,3.000000,2.000000,1.000000,2.000000,3.000000
75%,4.000000,3.000000,1.00000

In [5]:
datadf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 726 entries, 284 to 2
Data columns (total 78 columns):
job_title                         726 non-null category
job_taxo                          726 non-null category
job_id                            723 non-null category
job_conduct_res                   726 non-null int64
job_analyze_res                   726 non-null int64
job_buy_res_report                726 non-null int64
job_manage_res_proj               726 non-null int64
job_observe_res                   726 non-null int64
job_plan_res                      726 non-null int64
job_teach_res                     726 non-null int64
job_advocate_res                  726 non-null int64
job_hire_res_vendor               726 non-null int64
job_lead_res_team                 726 non-null int64
num_employees                     723 non-null category
num_researchers                   722 non-null category
primary_industry                  726 non-null category
types_res_used               

## 5. What is your company or organization's primary industry?

In [6]:
datadf.primary_industry = datadf.primary_industry.dropna().apply(nlp.basic_clean)
datadf.primary_industry = datadf.primary_industry.dropna().apply(nlp.lemmatize)

In [7]:
primary_industry_matrix, primary_industry_vector = nlp.create_wordcount_matrix(datadf.primary_industry.dropna(), ngram=(1,3), max_df=.5)
primary_industry_matrix, primary_industry_vector

(<726x180 sparse matrix of type '<class 'numpy.int64'>'
 	with 1239 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.5, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [8]:
lda5 = LatentDirichletAllocation(n_components= 3, random_state = 42)

In [9]:
lda5.fit(primary_industry_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=3, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [10]:
pyLDAvis.sklearn.prepare(lda5, primary_industry_matrix, primary_industry_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.210977 -0.162990       1        1  38.649390
2     -0.249731 -0.097304       2        1  33.882675
0      0.038754  0.260294       3        1  27.467936, topic_info=    Category       Freq                 Term      Total  loglift  logprob
157  Default  41.000000                 tech  41.000000  30.0000  30.0000
151  Default  40.000000             software  40.000000  29.0000  29.0000
75   Default  46.000000           healthcare  46.000000  28.0000  28.0000
67   Default  38.000000              fintech  38.000000  27.0000  27.0000
26   Default  35.000000           consulting  35.000000  26.0000  26.0000
159  Default  38.000000           technology  38.000000  25.0000  25.0000
51   Default  35.000000            education  35.000000  24.0000  24.0000
71   Default  28.000000           government  28.000000  23.0000  23.0000
86   Default  23.000000             industry  23.000000  22.0000  22.0000
65   Default  23.000000            financial  23.000000  21.0000  21.0000
142  Default  23.000000               retail  23.000000  20.0000  20.0000
143  Default  15.000000                 saas  15.000000  19.0000  19.0000
136  Default  21.000000             research  21.000000  18.0000  18.0000
23   Default  14.000000          consultancy  14.000000  17.0000  17.0000
92   Default  16.000000            insurance  16.000000  16.0000  16.0000
148  Default  37.000000              service  37.000000  15.0000  15.0000
66   Default  18.000000    financial service  18.000000  14.0000  14.0000
111  Default  15.000000               medium  15.000000  13.0000  13.0000
43   Default  14.000000              digital  14.000000  12.0000  12.0000
47   Default  15.000000            ecommerce  15.000000  11.0000  11.0000
118  Default  10.000000            nonprofit  10.000000  10.0000  10.0000
128  Default  10.000000               public  10.000000   9.0000   9.0000
55   Default  10.000000           enterprise  10.000000   8.0000   8.0000
146  Default   9.000000               sector   9.000000   7.0000   7.0000
63   Default  13.000000              finance  13.000000   6.0000   6.0000
39   Default  29.000000               design  29.000000   5.0000   5.0000
2    Default  15.000000               agency  15.000000   4.0000   4.0000
77   Default   9.000000               higher   9.000000   3.0000   3.0000
7    Default   9.000000              banking   9.000000   2.0000   2.0000
133  Default   9.000000                 real   9.000000   1.0000   1.0000
..       ...        ...                  ...        ...      ...      ...
55    Topic3   9.680832           enterprise  10.263468   1.2337  -3.5662
146   Topic3   8.727224               sector   9.322885   1.2261  -3.6699
107   Topic3   6.864622            marketing   7.448263   1.2106  -3.9100
74    Topic3   6.862561               health   7.448688   1.2102  -3.9103
56    Topic3   5.939852  enterprise software   6.511743   1.2002  -4.0547
42    Topic3   5.912632          development   6.509340   1.1960  -4.0593
50    Topic3   5.000976               edtech   5.573465   1.1838  -4.2267
160   Topic3   5.000658              telecom   5.573438   1.1837  -4.2268
129   Topic3   4.994836        public sector   5.572639   1.1827  -4.2279
154   Topic3   4.064626     software service   4.635434   1.1607  -4.4340
81    Topic3   4.064116                   hr   4.635395   1.1606  -4.4341
115   Topic3   4.062258             multiple   4.635292   1.1602  -4.4346
82    Topic3   3.126472              hr tech   3.697258   1.1245  -4.6964
97    Topic3   3.125312                legal   3.697160   1.1241  -4.6968
73    Topic3   3.124703             hardware   3.697110   1.1239  -4.6970
4     Topic3   3.120095           automotive   3.696885   1.1225  -4.6985
69    Topic3   3.111671            freelance   3.696387   1.1200  -4.7012
36    Topic3   2.188474                 data   2.759090   1.0

In [11]:
lda_W5 = lda5.transform(primary_industry_matrix)

In [12]:
top_doc_column5 = datadf.primary_industry.dropna()

In [13]:
word_count_matrix5, count_vect5 = nlp.create_wordcount_matrix(datadf.primary_industry, max_df=0.8, min_df=2, ngram=(1,3))

In [14]:
LDA5a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA5a.fit(word_count_matrix5)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [15]:
lda_H = LDA5a.transform(word_count_matrix5)

In [16]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column5, 3)

Top 3 Documents for Topic 0: 

Document 1
real estate tech

Document 2
real estate tech

Document 3
sotware development across domain such a data science deep learning health tech interaction etc

Top 3 Documents for Topic 1: 

Document 1
design research service and innovation planning aka design planning design ledstrategy

Document 2
market research agency 95 client are in financial service

Document 3
ux research we're a ux research agency a vendor used by fortune 500 company

Top 3 Documents for Topic 2: 

Document 1
commercial real estate

Document 2
commercial real estate

Document 3
architecture engineering construction aec industry manufacturing design medium entertainment industry

Top 3 Documents for Topic 3: 

Document 1
banking retail edu logistic government entertainment tourism aviation

Document 2
financial sector public sector resource technology retail healthcare

Document 3
aerospaceautomativebankingcentral government chemical consumer package goodsdefenceeducation ma

## 6. What types of research do you currently use to make decisions?

In [17]:
datadf.types_res_used = datadf.types_res_used.dropna().apply(nlp.basic_clean)
datadf.types_res_used = datadf.types_res_used.dropna().apply(nlp.lemmatize)

In [18]:
types_res_used_matrix, types_res_used_vector = nlp.create_wordcount_matrix(datadf.types_res_used.dropna(), ngram=(1,3), max_df=.5)
types_res_used_matrix, types_res_used_vector

(<690x1214 sparse matrix of type '<class 'numpy.int64'>'
 	with 8907 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.5, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [19]:
lda6 = LatentDirichletAllocation(n_components= 3, random_state = 42)

In [20]:
lda6.fit(types_res_used_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=3, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [21]:
pyLDAvis.sklearn.prepare(lda6, types_res_used_matrix, types_res_used_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.119819  0.097795       1        1  39.364890
1      0.039827 -0.137012       2        1  36.411622
0     -0.159646  0.039218       3        1  24.223488, topic_info=     Category        Freq                      Term       Total  loglift  \
728   Default  330.000000                  research  330.000000  30.0000   
660   Default  148.000000               qualitative  148.000000  29.0000   
694   Default   77.000000              quantitative   77.000000  28.0000   
67    Default   61.000000                      card   61.000000  27.0000   
836   Default   36.000000                      sort   36.000000  26.0000   
68    Default   35.000000                 card sort   35.000000  25.0000   
308   Default   49.000000                     group   49.000000  24.0000   
278   Default   49.000000                     focus   49.000000  23.0000   
866   Default  114.000000                     study  114.000000  22.0000   
279   Default   45.000000               focus group   45.000000  21.0000   
963   Default   85.000000                      test   85.000000  20.0000   
674   Default   27.000000  qualitative quantitative   27.000000  19.0000   
296   Default   42.000000                generative   42.000000  18.0000   
236   Default   40.000000                evaluative   40.000000  17.0000   
1147  Default  144.000000                      user  144.000000  16.0000   
388   Default  299.000000                 interview  299.000000  15.0000   
123   Default   44.000000        contextual inquiry   44.000000  14.0000   
369   Default   45.000000                   inquiry   45.000000  13.0000   
102   Default   42.000000                   concept   42.000000  12.0000   
502   Default   36.000000                    market   36.000000  11.0000   
1063  Default   25.000000                      tree   25.000000  10.0000   
503   Default   29.000000           market research   29.000000   9.0000   
679   Default   29.000000      qualitative research   29.000000   8.0000   
73    Default   25.000000              card sorting   25.000000   7.0000   
846   Default   25.000000                   sorting   25.000000   6.0000   
179   Default   48.000000                     diary   48.000000   5.0000   
120   Default   77.000000                contextual   77.000000   4.0000   
1107  Default   38.000000            usability test   38.000000   3.0000   
1160  Default   35.000000             user research   35.000000   2.0000   
531   Default   18.000000                     mixed   18.000000   1.0000   
...       ...         ...                       ...         ...      ...   
671    Topic3    4.632358        qualitative method    5.305093   1.2822   
694    Topic3   67.530177              quantitative   77.748936   1.2769   
531    Topic3   16.221072                     mixed   18.581512   1.2820   
1063   Topic3   21.436323                      tree   25.620144   1.2396   
532    Topic3   13.478587              mixed method   15.930781   1.2507   
73     Topic3   20.380764              card sorting   25.623911   1.1889   
846    Topic3   20.380286                   sorting   25.623915   1.1889   
660    Topic3   99.956044               qualitative  148.318580   1.0232   
992    Topic3    5.391227      testing card sorting    6.189601   1.2797   
983    Topic3    8.644947               test survey   10.621216   1.2120   
1088   Topic3    6.023079     unmoderated usability    7.073609   1.2571   
963    Topic3   49.441458                      test   85.915391   0.8653   
1107   Topic3   22.149119            usability test   38.122070   0.8749   
648    Topic3   14.075466                 prototype   22.092097   0.9671   
511    Topic3   25.662206                    method   53.022142   0.6922   
538    Topic3   20.784167                 moderated   39.881712   0.7661   
388    Topic3   70.086234              

In [22]:
lda_W6 = lda6.transform(types_res_used_matrix)

In [23]:
top_doc_column6 = datadf.types_res_used.dropna()

In [106]:
word_count_matrix6, count_vect6 = nlp.create_wordcount_matrix(datadf.types_res_used.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [107]:
LDA6a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA6a.fit(word_count_matrix6)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [108]:
lda_H = LDA6a.transform(word_count_matrix6)

In [109]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column6, 3)

Top 3 Documents for Topic 0: 

Document 1
qualitative interview survey card sort prototype testing usability testing analytics etc

Document 2
usability testing moderated and unmoderated card sort tree jacking max diff jtbd odi survey observational ethnographic research concept validation others a needed

Document 3
usability testing empathy1x1 interview survey card sort tree test prototype and concept testing diary study eye tracking

Top 3 Documents for Topic 1: 

Document 1
user research participant observation interview contextual enquiry usability testing feedback analysis data analysis survey

Document 2
contextual inquiry interview survey feedback form quant behavior analysis quant usability study usability testing ux workshop

Document 3
foundational product research usability testing 11 interview unmoderated user testing using usertestingcom survey contextual inquiry diary study analytics

Top 3 Documents for Topic 2: 

Document 1
qual research interview in lab moderated usabi

## 7. What types of research are you considering in the future?

In [28]:
datadf.future_res = datadf.future_res.dropna().apply(nlp.basic_clean)
datadf.future_res = datadf.future_res.dropna().apply(nlp.lemmatize)

In [29]:
future_res_matrix, future_res_vector = nlp.create_wordcount_matrix(datadf.future_res.dropna(), ngram=(1,3), max_df=.5)
future_res_matrix, future_res_vector

(<609x540 sparse matrix of type '<class 'numpy.int64'>'
 	with 3008 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.5, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [30]:
lda7 = LatentDirichletAllocation(n_components= 3, random_state = 42)

In [31]:
lda7.fit(future_res_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=3, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [32]:
pyLDAvis.sklearn.prepare(lda7, future_res_matrix, future_res_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.096808  0.147788       1        1  38.557449
2     -0.188419  0.002743       2        1  30.852486
1      0.091610 -0.150531       3        1  30.590065, topic_info=    Category        Freq                      Term       Total  loglift  \
371  Default  139.000000                  research  139.000000  30.0000   
434  Default   69.000000                     study   69.000000  29.0000   
90   Default   39.000000                     diary   39.000000  28.0000   
459  Default   91.000000                   testing   91.000000  27.0000   
91   Default   35.000000               diary study   35.000000  26.0000   
360  Default   29.000000                    remote   29.000000  25.0000   
12   Default   31.000000                 analytics   31.000000  24.0000   
255  Default   46.000000                    method   46.000000  23.0000   
343  Default   36.000000              quantitative   36.000000  22.0000   
501  Default   53.000000                 usability   53.000000  21.0000   
514  Default   38.000000                      user   38.000000  20.0000   
275  Default   17.000000                       new   17.000000  19.0000   
204  Default   29.000000                 interview   29.000000  18.0000   
0    Default   20.000000                        ab   20.000000  17.0000   
2    Default   18.000000                ab testing   18.000000  16.0000   
491  Default   13.000000                      type   13.000000  15.0000   
40   Default   15.000000                      card   15.000000  14.0000   
506  Default   32.000000         usability testing   32.000000  13.0000   
260  Default   10.000000                     mixed   10.000000  12.0000   
336  Default   27.000000                     quant   27.000000  11.0000   
444  Default   37.000000                    survey   37.000000  10.0000   
137  Default   13.000000                     focus   13.000000   9.0000   
494  Default   29.000000               unmoderated   29.000000   8.0000   
322  Default   16.000000                   product   16.000000   7.0000   
407  Default    9.000000                   session    9.000000   6.0000   
347  Default    9.000000     quantitative research    9.000000   5.0000   
162  Default   12.000000                     group   12.000000   4.0000   
481  Default   11.000000                  tracking   11.000000   3.0000   
194  Default   16.000000                   inquiry   16.000000   2.0000   
261  Default    8.000000              mixed method    8.000000   1.0000   
..       ...         ...                       ...         ...      ...   
496   Topic3    3.642591         unmoderated study    4.248432   1.0306   
234   Topic3    2.829431                        ll    3.398944   1.0011   
223   Topic3    2.829059               large scale    3.398942   1.0010   
128   Topic3    2.827574               eyetracking    3.398974   1.0004   
369   Topic3    2.827298  remote usability testing    3.398986   1.0003   
56    Topic3    2.825341           concept testing    3.399011   0.9996   
68    Topic3    2.821642            cultural probe    3.399077   0.9983   
201   Topic3    4.675913                 intercept    5.957531   0.9423   
404   Topic3    6.156312                   service    8.521886   0.8593   
106   Topic3   12.972301              ethnographic   21.318996   0.6877   
405   Topic3    4.455688            service design    5.960862   0.8935   
536   Topic3    7.511584                      work   11.882292   0.7259   
255   Topic3   20.482438                    method   46.522707   0.3641   
239   Topic3    7.308851                   looking   11.861830   0.7003   
494   Topic3   12.740237               unmoderated   29.122354   0.3578   
330   Topic3   11.829961               qualitative   26.334642   0.3842   
111   Topic3    8.897101               ethnography   17.844070   0.4885   
10    Topic3

In [33]:
lda_W7 = lda7.transform(future_res_matrix)

In [39]:
top_doc_column7 = datadf.future_res.dropna()

In [102]:
word_count_matrix7, count_vect7 = nlp.create_wordcount_matrix(datadf.future_res.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [103]:
LDA7a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA7a.fit(word_count_matrix7)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [104]:
lda_H = LDA7a.transform(word_count_matrix7)

In [105]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column7, 3)


Top 3 Documents for Topic 0: 

Document 1
user experience research workshop survey interview usability testing benchmark card sorting etc and i would like to delve deeper in analytics

Document 2
doing more strategic piece to help inform the product strategy early on before the okrs are set have fundamental understanding our user in different market since we have a global product and we havent done a lot of thinking around localized content and market difference doing information architecture research

Document 3
abn multivariate testing analytic research behavioural research card sortingtree testing content auditing experience mapping information architecture ia lab testing build persona development research participant recruitment management scenariotask mapping survey design analysis usability testing userstakeholder interview workshop

Top 3 Documents for Topic 1: 

Document 1
much more usability testing heatmappingbehavior tracking along the line of what pendoio offer exploratory 

## 10. Describe your educational background with research

In [45]:
datadf.research_educ_desc = datadf.research_educ_desc.dropna().apply(nlp.basic_clean)
datadf.research_educ_desc = datadf.research_educ_desc.dropna().apply(nlp.lemmatize)

In [46]:
research_educ_desc_matrix, research_educ_desc_vector = nlp.create_wordcount_matrix(datadf.research_educ_desc.dropna(), ngram=(1,3), max_df=.5)
research_educ_desc_matrix, research_educ_desc_vector

(<540x1119 sparse matrix of type '<class 'numpy.int64'>'
 	with 6861 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.5, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [47]:
lda10 = LatentDirichletAllocation(n_components= 3, random_state = 42)

In [48]:
lda10.fit(research_educ_desc_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=3, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [56]:
pyLDAvis.sklearn.prepare(lda10, research_educ_desc_matrix, research_educ_desc_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.032913 -0.095902       1        1  38.304096
2     -0.084885  0.071310       2        1  33.869446
0      0.117797  0.024591       3        1  27.826457, topic_info=     Category        Freq                       Term       Total  loglift  \
235   Default  102.000000                     degree  102.000000  30.0000   
594   Default   40.000000              master degree   40.000000  29.0000   
590   Default  131.000000                     master  131.000000  28.0000   
478   Default   22.000000                information   22.000000  27.0000   
871   Default   46.000000                    science   46.000000  26.0000   
738   Default   31.000000                    program   31.000000  25.0000   
334   Default   17.000000               ethnographic   17.000000  24.0000   
692   Default   76.000000                        phd   76.000000  23.0000   
493   Default   30.000000                interaction   30.000000  22.0000   
402   Default   29.000000                       grad   29.000000  21.0000   
258   Default  160.000000                     design  160.000000  20.0000   
167   Default   12.000000                   computer   12.000000  19.0000   
862   Default   21.000000                    running   21.000000  18.0000   
752   Default   71.000000                 psychology   71.000000  17.0000   
995   Default   35.000000                 took class   35.000000  16.0000   
453   Default   11.000000              humancomputer   11.000000  15.0000   
454   Default   11.000000  humancomputer interaction   11.000000  14.0000   
205   Default   90.000000                     course   90.000000  13.0000   
403   Default   24.000000                grad school   24.000000  12.0000   
187   Default   26.000000                 conference   26.000000  11.0000   
351   Default   13.000000    experimental psychology   13.000000  10.0000   
36    Default   34.000000               anthropology   34.000000   9.0000   
349   Default   19.000000               experimental   19.000000   8.0000   
404   Default   26.000000                   graduate   26.000000   7.0000   
344   Default   21.000000                 experience   21.000000   6.0000   
679   Default   10.000000       participated running   10.000000   5.0000   
865   Default   10.000000              running study   10.000000   4.0000   
962   Default   14.000000                     survey   14.000000   3.0000   
678   Default   25.000000               participated   25.000000   2.0000   
1106  Default   22.000000                   workshop   22.000000   1.0000   
...       ...         ...                        ...         ...      ...   
130    Topic3    6.746351         class participated    8.574385   1.0394   
862    Topic3   15.577793                    running   21.419573   0.9607   
493    Topic3   20.799852                interaction   30.043451   0.9115   
53     Topic3    8.412247                       area   11.132659   0.9990   
835    Topic3    5.942117             research study    7.720884   1.0173   
590    Topic3   60.486858                     master  131.154184   0.5052   
72     Topic3    6.988478            bachelor degree    9.446720   0.9778   
402    Topic3   17.134022                       grad   29.356793   0.7407   
403    Topic3   14.316077                grad school   24.168648   0.7555   
995    Topic3   18.685142                 took class   35.450192   0.6388   
945    Topic3   31.491616                      study   75.246889   0.4081   
330    Topic3   10.293692                engineering   16.317283   0.8185   
904    Topic3   12.894573                  sociology   23.281254   0.6883   
994    Topic3   28.566191                       took   78.048665   0.2741   
116    Topic3   40.067124                      class  129.445176   0.1065   
798    Topic3   73.225298                   research  325.379789  -0

In [49]:
lda_W10 = lda10.transform(research_educ_desc_matrix)

In [50]:
top_doc_column10 = datadf.research_educ_desc.dropna()

In [98]:
word_count_matrix10, count_vect10 = nlp.create_wordcount_matrix(datadf.research_educ_desc.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [99]:
LDA10a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA10a.fit(word_count_matrix10)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [100]:
lda_H = LDA10a.transform(word_count_matrix10)

In [101]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column10, 3)

Top 3 Documents for Topic 0: 

Document 1
i took professional development course in research up to one year in length at undergraduate level and then started postgraduate level private tuition i'd love to do a master or phd degree but am not able to manage the commitment even parttime alongside working full time i continue to take short course to increase my knowledge and develop my skill

Document 2
phd in human computer interaction research method data analysis coursework in various area eg ethnography statistic a well a supporting other people's research and eventually running my own research with supervision and advice from more senior people

Document 3
undergraduate degree in psychology extensive study of behavioral and scientific research methodology and analysis qual and quant interned and worked in lab for several year genetics neuroethology biopsychology graduate degree in architecture extensive study and practice of many type of research method

Top 3 Documents for Topic 1: 

## 14. How do you decide which events to attend?

In [57]:
datadf.how_pick_events = datadf.how_pick_events.dropna().apply(nlp.basic_clean)
datadf.how_pick_events = datadf.how_pick_events.dropna().apply(nlp.lemmatize)

In [58]:
how_pick_events_matrix, how_pick_events_vector = nlp.create_wordcount_matrix(datadf.how_pick_events.dropna(), ngram=(1,3), max_df=.5)
how_pick_events_matrix, how_pick_events_vector

(<644x1332 sparse matrix of type '<class 'numpy.int64'>'
 	with 7899 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.5, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [59]:
lda14 = LatentDirichletAllocation(n_components= 3, random_state = 42)

In [60]:
lda14.fit(how_pick_events_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=3, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [68]:
pyLDAvis.sklearn.prepare(lda14, how_pick_events_matrix, how_pick_events_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.012325 -0.093726       1        1  44.138342
1     -0.100731  0.054968       2        1  28.877657
0      0.113057  0.038758       3        1  26.984001, topic_info=     Category        Freq           Term       Total  loglift  logprob
1155  Default  165.000000          topic  165.000000  30.0000  30.0000
206   Default  194.000000     conference  194.000000  29.0000  29.0000
848   Default   60.000000            pay   60.000000  28.0000  28.0000
1136  Default   85.000000           time   85.000000  27.0000  27.0000
410   Default  194.000000          event  194.000000  26.0000  26.0000
105   Default   45.000000          based   45.000000  25.0000  25.0000
1158  Default   24.000000  topic covered   24.000000  24.0000  24.0000
1064  Default   92.000000        speaker   92.000000  23.0000  23.0000
976   Default   33.000000       relevant   33.000000  22.0000  22.0000
784   Default   23.000000     networking   23.000000  21.0000  21.0000
1324  Default   51.000000           year   51.000000  20.0000  20.0000
292   Default   29.000000        covered   29.000000  19.0000  19.0000
262   Default  145.000000           cost  145.000000  18.0000  18.0000
684   Default   42.000000          local   42.000000  17.0000  17.0000
973   Default   13.000000      relevance   13.000000  16.0000  16.0000
861   Default   31.000000         people   31.000000  15.0000  15.0000
819   Default   28.000000    opportunity   28.000000  14.0000  14.0000
1190  Default  135.000000         travel  135.000000  13.0000  13.0000
943   Default   18.000000        quality   18.000000  12.0000  12.0000
661   Default   42.000000           like   42.000000  11.0000  11.0000
646   Default   46.000000          learn   46.000000  10.0000  10.0000
717   Default   38.000000           look   38.000000   9.0000   9.0000
926   Default   12.000000   professional   12.000000   8.0000   8.0000
656   Default   14.000000          level   14.000000   7.0000   7.0000
288   Default   18.000000          cover   18.000000   6.0000   6.0000
1179  Default   12.000000  topic speaker   12.000000   5.0000   5.0000
755   Default    9.000000           meet    9.000000   4.0000   4.0000
1001  Default   10.000000          right   10.000000   3.0000   3.0000
1073  Default   13.000000  speaker topic   13.000000   2.0000   2.0000
148   Default   18.000000         career   18.000000   1.0000   1.0000
...       ...         ...            ...         ...      ...      ...
1158   Topic3   17.679055  topic covered   24.643959   0.9778  -4.8600
322    Topic3    9.146097       decision   11.862016   1.0499  -5.5190
1073   Topic3    9.996780  speaker topic   13.581145   1.0035  -5.4301
1155   Topic3   87.186763          topic  165.340153   0.6700  -3.2643
292    Topic3   19.671022        covered   29.780703   0.8952  -4.7532
1136   Topic3   47.873731           time   85.333908   0.7319  -3.8638
105    Topic3   27.529307          based   45.085438   0.8166  -4.4171
976    Topic3   21.138114       relevant   33.096431   0.8616  -4.6813
305    Topic3    7.056102           date    9.325534   1.0311  -5.7785
288    Topic3   11.800035          cover   18.777216   0.8454  -5.2642
262    Topic3   55.921340           cost  145.567386   0.3532  -3.7084
483    Topic3    8.501888            fit   12.770025   0.9031  -5.5921
1190   Topic3   49.585571         travel  135.430761   0.3052  -3.8287
1301   Topic3   27.961328           work   79.997973   0.2587  -4.4015
96     Topic3    9.654180           away   16.151547   0.7953  -5.4650
133    Topic3   21.529246         budget   57.682106   0.3244  -4.6629
191    Topic3   18.618593        company   47.259649   0.3784  -4.8082
1064   Topic3   27.510428        speaker   92.829190   0.0937  -4.4178
770    Topic3   10.821348          money   20.584370   0.6669  -5.3508
1151   Topic3   11.564939         timing   25.732597   0.5101  -5.

In [61]:
lda_W14 = lda14.transform(how_pick_events_matrix)

In [62]:
top_doc_column14 = datadf.how_pick_events.dropna()

In [94]:
word_count_matrix14, count_vect14 = nlp.create_wordcount_matrix(datadf.how_pick_events.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [95]:
LDA14a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA14a.fit(word_count_matrix14)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [96]:
lda_H = LDA14a.transform(word_count_matrix14)

In [97]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column14, 3)

Top 3 Documents for Topic 0: 

Document 1
it is usually a combination of date and my availability to travel the location the size the speaker list and whether the event seems likely to cater to more senior researcher or designer i'm at a point where traveling far for 101 level talk is just not useful to me

Document 2
depends on the cost of the event eg is it an evening meetup v a conference if it's the latter then it would depend on company training budget or depending on how relevant the topic are for my personal development goal also depending on the talk itself eg what the topic are which company are presenting i am more interested in advanced topic around strategy prioritisation or maybe new method

Document 3
location is the primary factor to consider because it will indicate whether i can afford to attend or not my company will cover cost of entry to a conference or workshop but will not cover travel or accommodation i also use personal vacation time to attend learning session s

## 15. What was the best professional learning experience you've ever had?  What made it great?

In [69]:
datadf.best_event = datadf.best_event.dropna().apply(nlp.basic_clean)
datadf.best_event = datadf.best_event.dropna().apply(nlp.lemmatize)

In [70]:
best_event_matrix, best_event_vector = nlp.create_wordcount_matrix(datadf.best_event.dropna(), ngram=(1,3), max_df=.5)
best_event_matrix, best_event_vector

(<594x1631 sparse matrix of type '<class 'numpy.int64'>'
 	with 9198 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.5, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [71]:
lda15 = LatentDirichletAllocation(n_components= 3, random_state = 42)

In [72]:
lda15.fit(best_event_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=3, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [ ]:
pyLDAvis.sklearn.prepare(lda15, best_event_matrix, best_event_vector)

In [73]:
lda_W15 = lda15.transform(best_event_matrix)

In [86]:
top_doc_column15 = datadf.best_event.dropna()

In [90]:
word_count_matrix15, count_vect15 = nlp.create_wordcount_matrix(datadf.best_event.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [91]:
LDA15a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA15a.fit(word_count_matrix15)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [92]:
lda_H = LDA15a.transform(word_count_matrix15)

In [93]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column15, 3)

Top 3 Documents for Topic 0: 

Document 1
two polar opposite experience c2mtl highly produced inspiring talk specific networking and meeting discussion and opportunity in an adjacent area to my own not my core sme so not the same old people i used what i learned from that experience to design an incredible offsite for my organization a few month after i returned a 2day immersive workshopclass on analyzing data that i took from iit many year ago it wa full of tool some of which were new to me and handson feedback not only did i walk away with tool and skill that i didn't have before i used that experience to design immersive learning course in the future it wa one of the earliest learnbydoing class that i'd taken

Document 2
the best experience i had not ux focussed wa the impact factory training workshop that wa over 2 day called 'train the trainer' fantastic and engaging trainer and really interactive session with no slide deck at all i liked the ucd london conference in 2010 or 2011 

## 16. What if any events have you attended on the subject of research in the past few years?

In [88]:
datadf.events_attend_recent = datadf.events_attend_recent.dropna().apply(nlp.basic_clean)
datadf.events_attend_recent = datadf.events_attend_recent.dropna().apply(nlp.lemmatize)

AttributeError: 'int' object has no attribute 'lower'

In [ ]:
lda16 = LatentDirichletAllocation(n_components= 3, random_state = 42)

In [ ]:
lda16.fit(events_attend_recent_matrix)

In [ ]:
pyLDAvis.sklearn.prepare(lda16, events_attend_recent_matrix, events_attend_recent_vector)

In [ ]:
lda_W16 = lda16.transform(events_attend_recent_matrix)

In [ ]:
top_doc_column16 = datadf.events_attend_recent.dropna()

In [ ]:
word_count_matrix16, count_vect16 = nlp.create_wordcount_matrix(datadf.events_attend_recent.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [ ]:
LDA16a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA16a.fit(word_count_matrix2)

In [ ]:
lda_H = LDA16a.transform(word_count_matrix2)

In [ ]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column16a, 3)

## 20. Did we miss any other types of conference sessions that you'd like to mention?

In [ ]:
datadf.other_conference_types = datadf.other_conference_types.dropna().apply(nlp.basic_clean)
datadf.other_conference_types = datadf.other_conference_types.dropna().apply(nlp.lemmatize)

In [ ]:
other_conference_types_matrix, other_conference_types_vector = nlp.create_wordcount_matrix(datadf.other_conference_types.dropna(), ngram=(1,3), max_df=.5)
other_conference_types_matrix, other_conference_types_vector

In [ ]:
lda20 = LatentDirichletAllocation(n_components= 3, random_state = 42)

In [ ]:
lda20.fit(other_conference_types_matrix)

In [ ]:
pyLDAvis.sklearn.prepare(lda20, other_conference_types_matrix, other_conference_types_vector)

In [ ]:
lda_W20 = lda20.transform(other_conference_types_matrix)

In [ ]:
top_doc_column = datadf.primary_industry.dropna()

In [ ]:
word_count_matrix2, count_vect2 = nlp.create_wordcount_matrix(datadf.primary_industry.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [ ]:
LDA2 = LatentDirichletAllocation(n_components=5, random_state=42)
LDA2.fit(word_count_matrix2)

In [ ]:
lda_H = LDA2.transform(word_count_matrix2)

In [ ]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column, 3)

## 22. If attending a conference about research, who might you be excited to see there?

In [ ]:
datadf.ideal_attendees = datadf.ideal_attendees.dropna().apply(nlp.basic_clean)
datadf.ideal_attendees = datadf.ideal_attendees.dropna().apply(nlp.lemmatize)

In [ ]:
ideal_att_matrix, ideal_att_vector = nlp.create_wordcount_matrix(datadf.ideal_attendees.dropna(), ngram=(1,3), max_df=.5)
ideal_att_matrix, ideal_att_vector

In [ ]:
lda22 = LatentDirichletAllocation(n_components= 3, random_state = 42)

In [ ]:
lda22.fit(ideal_att_matrix)

In [ ]:
pyLDAvis.sklearn.prepare(lda22, ideal_att_matrix, ideal_att_vector)

In [ ]:
lda_W22 = lda22.transform(ideal_att_matrix)

In [ ]:
top_doc_column = datadf.primary_industry.dropna()

In [ ]:
word_count_matrix2, count_vect2 = nlp.create_wordcount_matrix(datadf.primary_industry.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

LDA2 = LatentDirichletAllocation(n_components=5, random_state=42)
LDA2.fit(word_count_matrix2)

In [ ]:
lda_H = LDA2.transform(word_count_matrix2)

In [ ]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column, 3)